In [1]:
from google.colab import files 
uploaded = files.upload()

Saving Churn_Modelling (1).csv to Churn_Modelling (1).csv


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import io

#  Import and read the charity_data.csv.
df_churn = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling (1).csv']))
df_churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df_churn =df_churn.drop(["CustomerId", "Surname","RowNumber"], axis= 1)
df_churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Determine the number of unique values in each column.
df_churn.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

In [5]:
# Generate our categorical variable lists
churn_cat = df_churn.dtypes[df_churn.dtypes == "object"].index.tolist()

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_churn[churn_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(churn_cat)
encode_df.head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
df_churn = df_churn.merge(encode_df,left_index=True,right_index=True)
df_churn = df_churn.drop(churn_cat,1)
df_churn.head()

<ipython-input-7-38ab8f612fe5>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_churn = df_churn.drop(churn_cat,1)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0,1.0,0.0
1,608,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0,1.0,0.0
2,502,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0,1.0,0.0
3,699,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0,1.0,0.0
4,850,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0,1.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = df_churn["Exited"].values
X = df_churn.drop(["Exited"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

<ipython-input-8-789461e9adcf>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = df_churn.drop(["Exited"], 1).values


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layers1 = 100
hidden_nodes_layers2 = 80
hidden_nodes_layers3 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1400      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 40)                3240      
                                                                 
 dense_3 (Dense)             (None, 1)                 41        
                                                                 
Total params: 12,761
Trainable params: 12,761
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Import checkpoint dependencies 
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Streaming output truncated to the last 5000 lines.

Epoch 48: saving model to checkpoints/weights.48.hdf5
225/235 [===========================>..] - ETA: 0s - loss: 0.1750 - accuracy: 0.9276
Epoch 48: saving model to checkpoints/weights.48.hdf5
233/235 [============================>.] - ETA: 0s - loss: 0.1756 - accuracy: 0.9272
Epoch 48: saving model to checkpoints/weights.48.hdf5
235/235 [==============================] - 2s 8ms/step - loss: 0.1772 - accuracy: 0.9265
Epoch 49/100
  1/235 [..............................] - ETA: 1s - loss: 0.1621 - accuracy: 0.9062
Epoch 49: saving model to checkpoints/weights.49.hdf5
  9/235 [>.............................] - ETA: 1s - loss: 0.1841 - accuracy: 0.9167
Epoch 49: saving model to checkpoints/weights.49.hdf5

Epoch 49: saving model to checkpoints/weights.49.hdf5
 15/235 [>.............................] - ETA: 1s - loss: 0.1774 - accuracy: 0.9229
Epoch 49: saving model to checkpoints/weights.49.hdf5
 23/235 [=>............................] -

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 1.1926 - accuracy: 0.8092 - 249ms/epoch - 3ms/step
Loss: 1.1925729513168335, Accuracy: 0.8091999888420105


In [16]:
#Export the model to HDF5 file
nn.save("churn_project.h5")

In [20]:
from google.colab import files
files.download("churn_project.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>